# LambdaMARTを使用したランキング学習モデル

年度パックNPZファイルからデータを読み込み、LambdaMARTモデルを学習します。

## LambdaMARTとは

LambdaMARTはLambdaRankの改良版で、勾配ブースティングを使用したランキング学習アルゴリズムです。

### LambdaRankとの違い
- **より複雑なパターンを学習**: `num_leaves`を大きく設定
- **より慎重な学習**: `learning_rate`を低く設定
- **正則化の強化**: L1/L2正則化を追加
- **より多くのラウンド**: より多くのブーストラウンドで学習

### 期待される効果
- より高い的中率
- より安定した予測
- より複雑な特徴量の関係性を学習


## インポート


In [11]:
import sys
from pathlib import Path
import importlib

# プロジェクトルート（apps/prediction/）をパスに追加
project_root = Path().resolve().parent  # notebooks/ -> apps/prediction/
sys.path.insert(0, str(project_root))

# モジュールの再読み込みを確実にする
if 'src.preprocessor' in sys.modules:
    importlib.reload(sys.modules['src.preprocessor'])
if 'src.data_loader' in sys.modules:
    importlib.reload(sys.modules['src.data_loader'])
if 'src.lambdamart_predictor' in sys.modules:
    importlib.reload(sys.modules['src.lambdamart_predictor'])
if 'src.features' in sys.modules:
    importlib.reload(sys.modules['src.features'])

import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loader import load_annual_pack_npz, load_multiple_npz_files
from src.preprocessor import Preprocessor
from src.lambdamart_predictor import LambdaMARTPredictor
from src.features import Features

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


## 設定


In [12]:
# NPZファイルが格納されているベースパス
BASE_PATH = Path('./data')  # apps/prediction/notebooks/data

# 使用するデータタイプ
# レース開始時点で利用可能なデータのみを含める
# オッズデータ（OZ, OW, OU, OT, OV）と払戻データ（HJC, HJB）は除外
DATA_TYPES = [
    'BAC',  # 番組データ（レース条件・出走馬一覧）
    'KYI',  # 競走馬データ（牧場先情報付き・最も詳細）
    'SED',  # 成績速報データ（過去の成績・前走データ抽出に使用）
    'UKC',  # 馬基本データ（血統登録番号・性別・生年月日・血統情報）
    'TYB',  # 直前情報データ（出走直前の馬の状態・当日予想に最重要）
]

# 使用する年度
YEARS = [2024]  # 必要に応じて複数年度を指定

# モデル保存パス
MODEL_PATH = Path('../models/lambdamart_model_v1.txt')
MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)

# ファイル名の確認（デバッグ用）
print(f"BASE_PATH: {BASE_PATH.absolute()}")
if BASE_PATH.exists():
    npz_files = list(BASE_PATH.glob('*.npz'))
    print(f"見つかったNPZファイル: {len(npz_files)}件")
    for f in npz_files[:5]:  # 最初の5件を表示
        print(f"  - {f.name}")
else:
    print(f"警告: {BASE_PATH} が存在しません")


BASE_PATH: /Users/soichiro/Dev/umayomi/apps/prediction/notebooks/data
見つかったNPZファイル: 10件
  - jrdb_npz_SEC_2024.npz
  - jrdb_npz_UKC_2024.npz
  - jrdb_npz_KYH_2024.npz
  - jrdb_npz_KYI_2024.npz
  - jrdb_npz_SED_2024.npz


## データ読み込みと前処理


In [13]:
# 前処理を実行
preprocessor = Preprocessor()
df = preprocessor.process(
    base_path=BASE_PATH,
    data_types=DATA_TYPES,
    years=YEARS,
    use_annual_pack=True
)

print(f"\n前処理完了: {len(df)}件のデータ")
print(f"レース数: {df.index.nunique() if df.index.name == 'race_key' else df['race_key'].nunique()}")
print(f"特徴量数: {len([c for c in df.columns if c not in ['race_key', 'rank']])}")


データ読み込み中...


データ読み込み:   0%|          | 0/5 [00:00<?, ?it/s]

読み込み完了: BAC - 合計 3454件


データ読み込み:  60%|██████    | 3/5 [00:00<00:00,  7.12it/s]

読み込み完了: KYI - 合計 47181件
読み込み完了: SED - 合計 54953件


...

警告: TYB の結合は未実装です
前走データ抽出中...
前走データ抽出中（並列処理: 11コア）...
統計特徴量計算中...


/Users/soichiro/Dev/umayomi/apps/prediction/src/preprocessor.py:858: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  rename_map = {
/Users/soichiro/Dev/umayomi/apps/prediction/src/preprocessor.py:863: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  }


BACデータ: 総数=3454件, 年月日有効=3454件
  bac_date_mapのエントリ数: 3454
SEDデータ: 着順0以下を除外=7332件 (総数=54953件)
rankマージ前: combined_df=261375件, sed_data=804776件
  combined_dfのrace_keyユニーク数: 3454
  sed_dataのrace_keyユニーク数: 3709
  race_key一致数: 3454
  combined_dfのみのrace_key数: 0
  sed_dataのみのrace_key数: 255


/Users/soichiro/Dev/umayomi/apps/prediction/src/preprocessor.py:1324: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  combined_df = self.convert_to_numeric(combined_df)


rankマージ後: 有効=259949件, 欠損=1981件
  rankが欠損しているレース数: 384レース
    レース 20240106_06_01_1_03:
      combined_dfの馬番: [1]
      sed_dataの馬番: [2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0]
      欠損件数: 3件
    レース 20240106_06_01_1_09:
      combined_dfの馬番: [5]
      sed_dataの馬番: [1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0]
      欠損件数: 7件
    レース 20240106_08_01_1_05:
      combined_dfの馬番: [15]
      sed_dataの馬番: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 16.0, 17.0, 18.0]
      欠損件数: 7件
    レース 20240107_06_01_2_01:
      combined_dfの馬番: [11]
      sed_dataの馬番: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0]
      欠損件数: 1件
    レース 20240107_06_01_2_03:
      combined_dfの馬番: [7]
      sed_dataの馬番: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0]
      欠損件数: 2件
数値変換中...
ラベルエンコーディング中...
データ型最適化中...
前処理済みデータを保存しました: data/preprocessed_data/BAC_KYI_SED_TYB_UKC_2024_annual_data.npz


## データ確認


In [14]:
# データの上位50件を表示（すべての列を表示）
print("\n" + "=" * 60)
print("データの上位50件:")
print("=" * 60)
# すべての列を表示するためのオプション設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
# テーブル形式で表示
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', 50):
    print(df.head(50))



データの上位50件:
                     LS指数順位  R  クラスコード  ゴール内外  ゴール差  ゴール順位 ダ適性コード   テン指数  \
race_key                                                                   
20240106_08_01_1_01     5.0  1    38.0    1.0  30.0   15.0      3  -21.6   
20240106_08_01_1_01     5.0  1    38.0    1.0  11.0    9.0      3   -7.8   
20240106_08_01_1_01     5.0  1    38.0    1.0  11.0    9.0      3   -7.8   
20240106_08_01_1_01     5.0  1    38.0    1.0  11.0    9.0      3   -7.8   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   
20240106_08_01_1_01     2.0  1    38.0    4.0   2.0    3.0      2  -16.9   


## データ分割


In [15]:
# 学習・検証データに分割（レース単位で時系列分割）
train_df, val_df = preprocessor.split(df, train_ratio=0.8)

print(f"学習データ: {len(train_df)}件 ({train_df.index.nunique() if train_df.index.name == 'race_key' else train_df['race_key'].nunique()}レース)")
print(f"検証データ: {len(val_df)}件 ({val_df.index.nunique() if val_df.index.name == 'race_key' else val_df['race_key'].nunique()}レース)")

# 着順の分布を確認
if 'rank' in train_df.columns:
    print("\n学習データの着順分布:")
    print(train_df['rank'].value_counts().sort_index())
if 'rank' in val_df.columns:
    print("\n検証データの着順分布:")
    print(val_df['rank'].value_counts().sort_index())


学習データ: 213654件 (2763レース)
検証データ: 48276件 (691レース)

学習データの着順分布:
rank
1.0     16222
2.0     18174
3.0     18050
4.0     17967
5.0     17663
6.0     16824
7.0     16195
8.0     15492
9.0     14420
10.0    13077
11.0    11504
12.0    10057
13.0     8559
14.0     7183
15.0     5417
16.0     3596
17.0      971
18.0      654
Name: count, dtype: int64

検証データの着順分布:
rank
1.0     3410
2.0     3673
3.0     3605
4.0     3748
5.0     3735
6.0     3563
7.0     3541
8.0     3412
9.0     3253
10.0    3060
11.0    2847
12.0    2638
13.0    2235
14.0    1908
15.0    1682
16.0    1179
17.0     261
18.0     174
Name: count, dtype: int64


## LambdaMARTモデルの学習


In [16]:
# LambdaMART予測器を初期化
lambdamart_predictor = LambdaMARTPredictor(train_df, val_df)

print("LambdaMARTモデルのパラメータ:")
print(f"  - num_leaves: {lambdamart_predictor.common_params['num_leaves']}")
print(f"  - max_depth: {lambdamart_predictor.common_params['max_depth']}")
print(f"  - learning_rate: {lambdamart_predictor.common_params['learning_rate']}")
print(f"  - lambda_l1: {lambdamart_predictor.common_params['lambda_l1']}")
print(f"  - lambda_l2: {lambdamart_predictor.common_params['lambda_l2']}")
print(f"  - feature_fraction: {lambdamart_predictor.common_params['feature_fraction']}")
print(f"  - bagging_fraction: {lambdamart_predictor.common_params['bagging_fraction']}")

print("\nモデル学習開始...")
print("  - early_stopping_rounds: 早期停止のラウンド数（検証データの改善が止まったら学習を停止）")
print("  - num_boost_round: 最大ブーストラウンド数（デフォルト: 2000）")
print("  - optuna_timeout: Optunaの最大実行時間（秒、デフォルト: None）")
print("  - optuna_n_trials: Optunaの最大試行回数（デフォルト: 100）")

# モデルを学習（LambdaMART用のパラメータ）
model = lambdamart_predictor.train(
    early_stopping_rounds=50,
    num_boost_round=2000,  # LambdaMARTはより多くのラウンドが必要
    optuna_n_trials=100  # より多くの試行回数
)

print("\nモデル学習完了")


[I 2025-11-10 12:12:14,628] A new study created in memory with name: no-name-5670f6b9-7a64-405d-9ea8-0dfad95b5e3b


LambdaMARTモデルのパラメータ:
  - num_leaves: 31
  - max_depth: -1
  - learning_rate: 0.1
  - lambda_l1: 0.0
  - lambda_l2: 0.0
  - feature_fraction: 1.0
  - bagging_fraction: 1.0

モデル学習開始...
  - early_stopping_rounds: 早期停止のラウンド数（検証データの改善が止まったら学習を停止）
  - num_boost_round: 最大ブーストラウンド数（デフォルト: 2000）
  - optuna_timeout: Optunaの最大実行時間（秒、デフォルト: None）
  - optuna_n_trials: Optunaの最大試行回数（デフォルト: 100）


min_child_samples, val_score: 0.405072: 100%|##########| 5/5 [00:05<00:00,  1.04s/it]

最適パラメータ: {'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [1, 2, 3], 'boosting_type': 'gbdt', 'random_state': 0, 'num_leaves': 6, 'max_depth': -1, 'learning_rate': 0.1, 'min_gain_to_split': 0.0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'feature_fraction': 0.948, 'bagging_fraction': 0.5501286373091325, 'bagging_freq': 3, 'deterministic': True, 'force_row_wise': True, 'num_threads': 12, 'max_bin': 255, 'verbose': -1, 'early_stopping_rounds': 50, 'feature_pre_filter': False, 'min_child_samples': 20}

モデル学習完了


## モデルの保存


In [17]:
# モデルを保存
model.save_model(str(MODEL_PATH))
print(f"モデルを保存しました: {MODEL_PATH}")


モデルを保存しました: ../models/lambdamart_model_v1.txt


## モデル評価


In [ ]:
# 検証データで予測
features = Features()
val_predictions = LambdaMARTPredictor.predict(model, val_df, features)

# rankが欠損している行を除外（評価用）
val_predictions_with_rank = val_predictions[val_predictions['rank'].notna()].copy()
print(f"rankが有効なデータ: {len(val_predictions_with_rank)}件")
print(f"rankが欠損しているデータ: {len(val_predictions) - len(val_predictions_with_rank)}件")

# シンプルな評価（オッズデータなし）
LambdaMARTPredictor.print_evaluation(val_predictions_with_rank)

# オッズデータがある場合の回収率計算（オプション）
# preprocessorからオッズデータを取得してマージ
try:
    # preprocessorが保持しているオッズデータを取得
    odds_df = preprocessor.get_odds_data()
    
    if odds_df is None or len(odds_df) == 0:
        print("\n警告: オッズデータが取得できませんでした。回収率は計算されません。")
        print("（SEDデータに'確定単勝オッズ'カラムが存在しない可能性があります）")
    else:
        print(f"\nオッズデータ取得: {len(odds_df)}件")
        
        # val_predictionsに確定単勝オッズをマージ
        # 注意: LambdaMARTPredictor.predict()でrace_keyと馬番の型は既に統一されている
        val_predictions_with_odds = val_predictions_with_rank.copy()
        
        # インデックスがrace_keyの場合はリセット
        if val_predictions_with_odds.index.name == 'race_key':
            val_predictions_with_odds = val_predictions_with_odds.reset_index()
        
        # オッズデータをマージ（race_keyと馬番の型は既に統一されている）
        val_predictions_with_odds = val_predictions_with_odds.merge(
            odds_df,
            on=['race_key', '馬番'],
            how='left'
        )
        
        # 確定単勝オッズカラムが存在するか確認
        if '確定単勝オッズ' not in val_predictions_with_odds.columns:
            print("警告: '確定単勝オッズ'カラムが見つかりません。")
            print("（回収率は計算されません）")
        else:
            # 確定単勝オッズを数値に変換
            def parse_odds(odds_str):
                if pd.isna(odds_str) or odds_str == '':
                    return np.nan
                try:
                    return float(str(odds_str).strip())
                except:
                    return np.nan
            
            val_predictions_with_odds['確定単勝オッズ'] = val_predictions_with_odds['確定単勝オッズ'].apply(parse_odds)
            
            # オッズデータがマージできたか確認
            odds_merged_count = val_predictions_with_odds['確定単勝オッズ'].notna().sum()
            print(f"オッズデータがマージできた件数: {odds_merged_count}/{len(val_predictions_with_odds)}")
            
            if odds_merged_count == 0:
                print("警告: オッズデータが1件もマージできませんでした。回収率は計算されません。")
            else:
                # 回収率を含む評価
                LambdaMARTPredictor.print_evaluation(val_predictions_with_odds, odds_col='確定単勝オッズ')
except Exception as e:
    print(f"\nオッズデータの取得に失敗しました: {e}")
    print("（回収率は計算されません）")
    import traceback
    traceback.print_exc()


rankが有効なデータ: 47924件
rankが欠損しているデータ: 352件

モデル評価結果

NDCG（Normalized Discounted Cumulative Gain）:
  NDCG@1: 0.3753
  NDCG@2: 0.3783
  NDCG@3: 0.3832

1着的中率: 24.60% (170/691レース)
3着以内的中率: 57.02% (394/691レース)

平均順位誤差: 36.33位
オッズデータ取得: 54953件

オッズデータの取得に失敗しました: '確定単勝オッズ'
（回収率は計算されません）


/var/folders/mm/c2gn__y56p58dssh1byn5wbh0000gn/T/ipykernel_29313/2867699574.py:35: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  val_predictions_with_odds = val_predictions_with_odds.merge(
Traceback (most recent call last):
  File "/Users/soichiro/Library/Python/3.9/lib/python/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: '確定単勝オッズ'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/va